In [1]:
import networkx as nx
import csv
import pprint as pp


###### graph creation

In [2]:
# graph creation function
def graph_create_from_tsv(tsvFilePath):
    # reading tsv file
    input_file_handler = open(tsvFilePath, 'r', encoding="utf-8")
    tsv_reader = csv.reader(input_file_handler, delimiter='\t')
    tsv_reader.__next__()
    list_adj = []
    # inserting adjacent pokemon couples in a list
    for pokemon in tsv_reader:
        pokemon1 = pokemon[0]
        pokemon2 = pokemon[1]
        list_adj.append((pokemon1, pokemon2))
    input_file_handler.close()

    #creating graph
    graph = nx.Graph()
    graph.add_edges_from(list_adj)
    return graph

###### top k computation definition

In [3]:
def compute_top_k(map__node_id__score, remove_pkmn, k=20):
    list__node_id__score = [(node_id, score) for node_id, score in map__node_id__score.items() if node_id not in remove_pkmn]
    list__node_id__score.sort(key=lambda x: (-x[1], x[0]))
    return list__node_id__score[:k]

###### creation of the teleporting probability distribution for the selected topic

In [4]:
def prob_topic(graph, poke_topic):
    num_nodes_topic = len(poke_topic)
    # creating dictionary with keys the nodes of the graph, and values set to 0
    d_topic_nodes = dict.fromkeys(list(graph.nodes), 0)
    for pokemon in poke_topic:
        d_topic_nodes[pokemon] = 1/num_nodes_topic
    return d_topic_nodes

##### starting code
##### first given sets
Set_A = set(["Pikachu"]) <br>
Set_B = set(["Venusaur", "Charizard", "Blastoise"]) <br>
Set_C = set(["Excadrill", "Dracovish", "Whimsicott", "Milotic"])

In [5]:
graph = graph_create_from_tsv('dataset/pkmn_graph_data.tsv')

team_size = 6

# given sets
Set_A = set(["Pikachu"])
Set_B = set(["Venusaur", "Charizard", "Blastoise"])
Set_C = set(["Excadrill", "Dracovish", "Whimsicott", "Milotic"])

sets = {'Set_A': Set_A,
        'Set_B': Set_B,
        'Set_C': Set_C
}
teams = {}

#### Computation of the PageRank vector for given sets

In [6]:
for key, val in sets.items():
    topic_pers = prob_topic(graph, val)
    map__node_id__node_pagerank_value = nx.pagerank(graph, alpha=0.33,
                                                    personalization=topic_pers)
    top_k__node_id__node_pagerank_value = compute_top_k(map__node_id__node_pagerank_value,
                                                        remove_pkmn=val,
                                                        k=team_size - len(val))
    pp.pprint(top_k__node_id__node_pagerank_value)
    teams['team_' + key[-1]] = set([i[0] for i in top_k__node_id__node_pagerank_value] + list(val))

[('Gengar', 0.016487640693345462),
 ('Dragonite', 0.016010002128851935),
 ("Sirfetch'd", 0.01599150628957732),
 ('Kingdra', 0.015953347602938955),
 ('Lucario', 0.015876479354420946)]
[('Dusclops', 0.008665975297740788),
 ('Urshifu', 0.008394906730815232),
 ('Torkoal', 0.006648512031539321)]
[('Tyranitar', 0.00731484051799319), ('Cinderace', 0.006973637903344642)]


#### Printing sets of Pokemons with respective input sets from A to C

In [7]:
print('Sets of Pokemons with Set_A')
print(teams['team_A'])
print('###########################')
print('Sets of Pokemons with Set_B')
print(teams['team_B'])
print('###########################')
print('Sets of Pokemons with Set_C')
print(teams['team_C'])
print('###########################')
print('###########################')

Sets of Pokemons with Set_A
{'Pikachu', 'Dragonite', 'Gengar', "Sirfetch'd", 'Lucario', 'Kingdra'}
###########################
Sets of Pokemons with Set_B
{'Charizard', 'Urshifu', 'Venusaur', 'Dusclops', 'Torkoal', 'Blastoise'}
###########################
Sets of Pokemons with Set_C
{'Dracovish', 'Cinderace', 'Tyranitar', 'Milotic', 'Whimsicott', 'Excadrill'}
###########################
###########################


#### Same code, but with different given sets than before

Set_1 = ["Charizard"] <br>
Set_2 = ["Venusaur"] <br>
Set_3 = ["Kingdra"] <br>
Set_4 = set(["Charizard", "Venusaur"]) <br>
Set_5 = set(["Charizard", "Kingdra"]) <br>
Set_6 = set(["Venusaur", "Kingdra"]) <br>

In [8]:
Set_1 = ["Charizard"]
Set_2 = ["Venusaur"]
Set_3 = ["Kingdra"]
Set_4 = set(["Charizard", "Venusaur"])
Set_5 = set(["Charizard", "Kingdra"])
Set_6 = set(["Venusaur", "Kingdra"])

sets = {'Set_1': Set_1,
        'Set_2': Set_2,
        'Set_3': Set_3,
        'Set_4': Set_4,
        'Set_5': Set_5,
        'Set_6': Set_6
}
teams = {}

In [9]:
for key, val in sets.items():
    topic_pers = prob_topic(graph, val)
    map__node_id__node_pagerank_value = nx.pagerank(graph, alpha=0.33,
                                                    personalization=topic_pers)
    top_k__node_id__node_pagerank_value = compute_top_k(map__node_id__node_pagerank_value,
                                                        remove_pkmn=val,
                                                        k=team_size - len(val))
    pp.pprint(top_k__node_id__node_pagerank_value)
    teams['team_' + key[-1]] = set([i[0] for i in top_k__node_id__node_pagerank_value] + list(val))

[('Torkoal', 0.009326141035662906),
 ('Venusaur', 0.009097957694151023),
 ('Clefairy', 0.009030034219149143),
 ('Groudon', 0.008980365368672534),
 ('Grimmsnarl', 0.008872171170580347)]
[('Torkoal', 0.009725189857536932),
 ('Dusclops', 0.008959547497020031),
 ('Charizard', 0.008794691285464033),
 ('Porygon2', 0.008640048209172064),
 ('Stakataka', 0.008629413684703267)]
[('Politoed', 0.010453137370905956),
 ('Kyogre', 0.010290391347309004),
 ('Tornadus', 0.009756473037832985),
 ('Togedemaru', 0.009727794858781478),
 ('Tsareena', 0.009580057893446523)]
[('Torkoal', 0.00952566544659992),
 ('Dusclops', 0.008760660793738552),
 ('Stakataka', 0.008748839693770511),
 ('Groudon', 0.008692699858267856)]
[('Raichu', 0.008709148693346727),
 ('Bronzong', 0.008558796603759267),
 ('Sableye', 0.008489253192342639),
 ('Politoed', 0.005584803157233736)]
[('Tapu Koko', 0.008638658279258127),
 ('Bronzong', 0.008480539740990126),
 ('Corviknight', 0.008476696218413945),
 ('Politoed', 0.005476971930695063)]


In [10]:
print('Sets of Pokemons with Charizard')
print(teams['team_1'])
print('###########################')
print('Sets of Pokemons with Venusaur')
print(teams['team_2'])
print('###########################')
print('Sets of Pokemons with Kingdra')
print(teams['team_3'])
print('###########################')
print('Sets of Pokemons with Charizard and Venusaur')
print(teams['team_4'])
print('###########################')
print('Sets of Pokemons with Charizard and Kingdra')
print(teams['team_5'])
print('###########################')
print('Sets of Pokemons with Venusaur and Kingdra')
print(teams['team_6'])
print('###########################')
print('###########################')

Sets of Pokemons with Charizard
{'Grimmsnarl', 'Charizard', 'Clefairy', 'Venusaur', 'Torkoal', 'Groudon'}
###########################
Sets of Pokemons with Venusaur
{'Charizard', 'Venusaur', 'Porygon2', 'Torkoal', 'Stakataka', 'Dusclops'}
###########################
Sets of Pokemons with Kingdra
{'Tornadus', 'Kyogre', 'Tsareena', 'Politoed', 'Togedemaru', 'Kingdra'}
###########################
Sets of Pokemons with Charizard and Venusaur
{'Charizard', 'Venusaur', 'Dusclops', 'Torkoal', 'Stakataka', 'Groudon'}
###########################
Sets of Pokemons with Charizard and Kingdra
{'Raichu', 'Charizard', 'Politoed', 'Bronzong', 'Sableye', 'Kingdra'}
###########################
Sets of Pokemons with Venusaur and Kingdra
{'Tapu Koko', 'Politoed', 'Bronzong', 'Venusaur', 'Kingdra', 'Corviknight'}
###########################
###########################


In [12]:
print('Number of team members inside the Team(Charizard, Venusaur) '
      'that are neither in Team(Charizard) nor in Team(Venusaur)')
print(len(teams['team_4'].difference(teams['team_1'].union(teams['team_2']))))
print('###########################')
print('Number of team members inside the Team(Charizard, Kingdra) '
      'that are neither in Team(Charizard) nor in Team(Kingdra)')
print(len(teams['team_5'].difference(teams['team_1'].union(teams['team_3']))))
print('###########################')
print('Number of team members inside the Team(Venusaur, Kingdra) '
      'that are neither in Team(Venusaur) nor in Team(Kingdra)')
print(len(teams['team_6'].difference(teams['team_2'].union(teams['team_3']))))
print('###########################')
print('###########################')

Number of team members inside the Team(Charizard, Venusaur) that are neither in Team(Charizard) nor in Team(Venusaur)
0
###########################
Number of team members inside the Team(Charizard, Kingdra) that are neither in Team(Charizard) nor in Team(Kingdra)
3
###########################
Number of team members inside the Team(Venusaur, Kingdra) that are neither in Team(Venusaur) nor in Team(Kingdra)
3
###########################
###########################
